In [88]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly.offline import iplot, plot
from empath import Empath

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import spacy

from sklearn.preprocessing import MinMaxScaler

In [100]:
df = pd.read_csv("data/Features_For_Traditional_ML_Techniques.csv")
df

,Unnamed: 0,majority_target,statement,BinaryNumTarget,tweet,followers_count,friends_count,favourites_count,statuses_count,listed_count,...,determiners,conjunctions,dots,exclamation,questions,ampersand,capitals,digits,long_word_freq,short_word_freq
0,0,True,End of eviction moratorium means millions of A...,1.0,@POTUS Biden Blunders - 6 Month Update\n\nInfl...,4262.0,3619.0,34945.0,16423.0,44.0,...,0,0,5,0,1,0,33,3,5,19
1,1,True,End of eviction moratorium means millions of A...,1.0,@S0SickRick @Stairmaster_ @6d6f636869 Not as m...,1393.0,1621.0,31436.0,37184.0,64.0,...,0,2,1,0,0,0,14,0,2,34
2,2,True,End of eviction moratorium means millions of A...,1.0,THE SUPREME COURT is siding with super rich pr...,9.0,84.0,219.0,1184.0,0.0,...,0,1,0,0,0,0,3,0,4,10
3,3,True,End of eviction moratorium means millions of A...,1.0,@POTUS Biden Blunders\n\nBroken campaign promi...,4262.0,3619.0,34945.0,16423.0,44.0,...,0,1,3,0,0,1,6,8,1,30
4,4,True,End of eviction moratorium means millions of A...,1.0,@OhComfy I agree. The confluence of events rig...,70.0,166.0,15282.0,2194.0,0.0,...,0,1,3,0,1,0,11,3,2,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134193,134193,False,Joe Bidens great-grandfather Joseph J. Biden w...,0.0,Joe Biden's family owned African slaves....\n\...,2075.0,2662.0,156306.0,62375.0,3.0,...,0,2,1,0,3,0,31,0,4,32
134194,134194,False,Joe Bidens great-grandfather Joseph J. Biden w...,0.0,"Joe Bidens great, great grandfather was a slav...",1636.0,1626.0,215290.0,44011.0,10.0,...,0,0,1,0,0,0,16,6,2,8
134195,134195,False,Joe Bidens great-grandfather Joseph J. Biden w...,0.0,"@ChevyChaseToGo ""Joe Bidens great-grandfather ...",286.0,2501.0,28377.0,7617.0,1.0,...,1,4,1,0,0,1,2,4,0,37
134196,134196,False,Joe Bidens great-grandfather Joseph J. Biden w...,0.0,@JoeBiden Facts are Bidens VP Kamala Harris Gr...,2.0,45.0,16.0,509.0,0.0,...,1,0,3,0,0,0,0,0,1,11


In [101]:
#Brisanje kolona

df = df.drop(columns=['Unnamed: 0'])
df = df.drop(columns=['following'])
df = df.drop(columns=['BinaryNumTarget'])
df

,majority_target,statement,tweet,followers_count,friends_count,favourites_count,statuses_count,listed_count,embeddings,BotScore,...,determiners,conjunctions,dots,exclamation,questions,ampersand,capitals,digits,long_word_freq,short_word_freq
0,True,End of eviction moratorium means millions of A...,@POTUS Biden Blunders - 6 Month Update\n\nInfl...,4262.0,3619.0,34945.0,16423.0,44.0,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]],0.03,...,0,0,5,0,1,0,33,3,5,19
1,True,End of eviction moratorium means millions of A...,@S0SickRick @Stairmaster_ @6d6f636869 Not as m...,1393.0,1621.0,31436.0,37184.0,64.0,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]],0.03,...,0,2,1,0,0,0,14,0,2,34
2,True,End of eviction moratorium means millions of A...,THE SUPREME COURT is siding with super rich pr...,9.0,84.0,219.0,1184.0,0.0,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]],0.03,...,0,1,0,0,0,0,3,0,4,10
3,True,End of eviction moratorium means millions of A...,@POTUS Biden Blunders\n\nBroken campaign promi...,4262.0,3619.0,34945.0,16423.0,44.0,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]],0.03,...,0,1,3,0,0,1,6,8,1,30
4,True,End of eviction moratorium means millions of A...,@OhComfy I agree. The confluence of events rig...,70.0,166.0,15282.0,2194.0,0.0,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]],0.03,...,0,1,3,0,1,0,11,3,2,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134193,False,Joe Bidens great-grandfather Joseph J. Biden w...,Joe Biden's family owned African slaves....\n\...,2075.0,2662.0,156306.0,62375.0,3.0,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]],0.03,...,0,2,1,0,3,0,31,0,4,32
134194,False,Joe Bidens great-grandfather Joseph J. Biden w...,"Joe Bidens great, great grandfather was a slav...",1636.0,1626.0,215290.0,44011.0,10.0,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]],0.03,...,0,0,1,0,0,0,16,6,2,8
134195,False,Joe Bidens great-grandfather Joseph J. Biden w...,"@ChevyChaseToGo ""Joe Bidens great-grandfather ...",286.0,2501.0,28377.0,7617.0,1.0,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]],0.03,...,1,4,1,0,0,1,2,4,0,37
134196,False,Joe Bidens great-grandfather Joseph J. Biden w...,@JoeBiden Facts are Bidens VP Kamala Harris Gr...,2.0,45.0,16.0,509.0,0.0,[[0 1 0 0 0 0 1 0 0 0 0 0 1 1 0 0 1]],0.03,...,1,0,3,0,0,0,0,0,1,11


In [102]:
df.shape

(134198, 61)

In [ ]:
#Brisemo kolone koje cemo ponovo racunati zbog naseg preprocesiranja i neke nepotrebne

columns_to_remove = [
    'ORG_percentage', 'NORP_percentage', 'GPE_percentage',
    'PERSON_percentage', 'MONEY_percentage', 'DATE_percentage',
    'CARDINAL_percentage', 'PERCENT_percentage', 'ORDINAL_percentage',
    'FAC_percentage', 'LAW_percentage', 'PRODUCT_percentage',
    'EVENT_percentage', 'TIME_percentage', 'LOC_percentage',
    'WORK_OF_ART_percentage', 'QUANTITY_percentage', 'LANGUAGE_percentage',
    'Word count', 'Max word length', 'Min word length',
    'Average word length', 'present_verbs', 'past_verbs', 'adjectives',
    'adverbs', 'adpositions', 'pronouns', 'TOs', 'determiners',
    'conjunctions', 'dots', 'exclamation', 'questions', 'ampersand',
    'capitals', 'digits', 'long_word_freq', 'short_word_freq'
]

df = df.drop(columns=columns_to_remove, errors="ignore") 


In [104]:
df.shape

(134198, 22)

# Text cleaning

In [105]:
def clean_text(text):
    if isinstance(text, str):  
        text = re.sub(r"@\w+", "", text)  # @pominjanja
        text = re.sub(r"http\S+|www\S+", "", text)  # URL
        text = re.sub(r"[^\w\s]|_", "", text) # specijalni karakteri (karakteri koji nisu slova ili brojevi)
        text = text.lower()  # konverzija u mala slova
        text = re.sub(r"\s+", " ", text).strip()  # visestruke bjeline
        return text
    return text  

df['tweet_new'] = df["tweet"].apply(clean_text)
df[["tweet", "tweet_new"]].head(5)

,tweet,tweet_new
0,@POTUS Biden Blunders - 6 Month Update\n\nInfl...,biden blunders 6 month update inflation delta ...
1,@S0SickRick @Stairmaster_ @6d6f636869 Not as m...,not as many people are literally starving and ...
2,THE SUPREME COURT is siding with super rich pr...,the supreme court is siding with super rich pr...
3,@POTUS Biden Blunders\n\nBroken campaign promi...,biden blunders broken campaign promises inflat...
4,@OhComfy I agree. The confluence of events rig...,i agree the confluence of events right now is ...


# Stop Word Removal 

In [107]:
#Koristimo stopwords nltk biblioteke

stop_words = set(stopwords.words("english"))

def remove_stopwords(text):
    if isinstance(text, str):
        words = text.split()  
        filtered_words = [word for word in words if word not in stop_words] 
        return " ".join(filtered_words) 
    return text  

df["tweet_new"] = df["tweet_new"].apply(remove_stopwords)
df[["tweet", "tweet_new"]].head(5)

,tweet,tweet_new
0,@POTUS Biden Blunders - 6 Month Update\n\nInfl...,biden blunders 6 month update inflation delta ...
1,@S0SickRick @Stairmaster_ @6d6f636869 Not as m...,many people literally starving streets 19th ce...
2,THE SUPREME COURT is siding with super rich pr...,supreme court siding super rich property owner...
3,@POTUS Biden Blunders\n\nBroken campaign promi...,biden blunders broken campaign promises inflat...
4,@OhComfy I agree. The confluence of events rig...,agree confluence events right unprecedented af...


# Lematizacija

In [108]:

nlp = spacy.load("en_core_web_sm")

def lemmatize_text_with_no_verbs(text):
    doc = nlp(text)
    lemmatized_text = []
    
    for token in doc:
        if token.pos_ != "VERB": 
            lemmatized_text.append(token.lemma_)
        else:
            lemmatized_text.append(token.text)  
    
    return " ".join(lemmatized_text)



df["tweet_new"] = df["tweet_new"].apply(lemmatize_text_with_no_verbs)
df[["tweet", "tweet_new"]].head(5)


,tweet,tweet_new
0,@POTUS Biden Blunders - 6 Month Update\n\nInfl...,biden blunder 6 month update inflation delta m...
1,@S0SickRick @Stairmaster_ @6d6f636869 Not as m...,many people literally starving street 19th cen...
2,THE SUPREME COURT is siding with super rich pr...,supreme court siding super rich property owner...
3,@POTUS Biden Blunders\n\nBroken campaign promi...,biden blunders broken campaign promises inflat...
4,@OhComfy I agree. The confluence of events rig...,agree confluence event right unprecedented afg...


# Tokenizacija

In [109]:

def tokenize_text(text):
    if isinstance(text, str):
        return word_tokenize(text)  
    return text  

df["tweet_tokens"] = df["tweet_new"].apply(tokenize_text)

df[["tweet_new", "tweet_tokens"]].head()

,tweet_new,tweet_tokens
0,biden blunder 6 month update inflation delta m...,"[biden, blunder, 6, month, update, inflation, ..."
1,many people literally starving street 19th cen...,"[many, people, literally, starving, street, 19..."
2,supreme court siding super rich property owner...,"[supreme, court, siding, super, rich, property..."
3,biden blunders broken campaign promises inflat...,"[biden, blunders, broken, campaign, promises, ..."
4,agree confluence event right unprecedented afg...,"[agree, confluence, event, right, unprecedente..."


In [110]:
print(df[df["tweet_new"].str.strip() == ""][["tweet", "tweet_new"]])

#Otkrili smo da postoje 3 nova prazna tvita

                                                    tweet tweet_new
20330                           @ThorJohnson_xxx Me me me          
20511                       @threesome_xxx Who are they!?          
104356  @oregon_resister @bamableu @Mackaveli8 @The_Mr...          


In [111]:
df = df[df['tweet_new'].str.strip() != ""]


In [112]:
df.shape


(134195, 24)

# Tekstualne karakteristike

In [113]:


nlp = spacy.load("en_core_web_sm")

def analyze_text(text):
    doc = nlp(text)
    
    word_count = len(doc)
    max_word_length = max(len(token.text) for token in doc) if word_count > 0 else 0
    min_word_length = min(len(token.text) for token in doc) if word_count > 0 else 0
    avg_word_length = sum(len(token.text) for token in doc) / word_count if word_count > 0 else 0

    verbs_present = sum(1 for token in doc if token.pos_ == "VERB" and token.tag_ == "VBZ")  
    verbs_past = sum(1 for token in doc if token.pos_ == "VERB" and token.tag_ == "VBD") 
    adjectives = sum(1 for token in doc if token.pos_ == "ADJ")
    adverbs = sum(1 for token in doc if token.pos_ == "ADV")
    pronouns = sum(1 for token in doc if token.pos_ == "PRON")
    conjunctions = sum(1 for token in doc if token.pos_ == "CCONJ")
    determiners = sum(1 for token in doc if token.pos_ == "DET")
    prepositions = sum(1 for token in doc if token.pos_ == "ADP")
    digits = sum(1 for token in doc if token.text.isdigit())



    stats = {
        'Word count': word_count,
        'Max word length': max_word_length,
        'Min word length': min_word_length,
        'Average word length': avg_word_length,
        'Present verbs': verbs_present,
        'Past verbs': verbs_past,
        'Adjectives': adjectives,
        'Adverbs': adverbs,
        'Pronouns': pronouns,
        'Conjunctions': conjunctions,
        'Determiners': determiners,
        'Prepositions': prepositions,
        'Digits': digits,
        
    }
    return stats

df_stats = df["tweet_new"].apply(analyze_text).apply(pd.Series)

df = pd.concat([df, df_stats], axis=1)

df.head()


,majority_target,statement,tweet,followers_count,friends_count,favourites_count,statuses_count,listed_count,embeddings,BotScore,...,Average word length,Present verbs,Past verbs,Adjectives,Adverbs,Pronouns,Conjunctions,Determiners,Prepositions,Digits
0,True,End of eviction moratorium means millions of A...,@POTUS Biden Blunders - 6 Month Update\n\nInfl...,4262.0,3619.0,34945.0,16423.0,44.0,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]],0.03,...,6.354839,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,2.0
1,True,End of eviction moratorium means millions of A...,@S0SickRick @Stairmaster_ @6d6f636869 Not as m...,1393.0,1621.0,31436.0,37184.0,64.0,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]],0.03,...,7.000000,0.0,0.0,4.0,2.0,0.0,0.0,0.0,0.0,0.0
2,True,End of eviction moratorium means millions of A...,THE SUPREME COURT is siding with super rich pr...,9.0,84.0,219.0,1184.0,0.0,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]],0.03,...,6.000000,0.0,0.0,6.0,2.0,0.0,0.0,0.0,0.0,0.0
3,True,End of eviction moratorium means millions of A...,@POTUS Biden Blunders\n\nBroken campaign promi...,4262.0,3619.0,34945.0,16423.0,44.0,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]],0.03,...,6.515152,2.0,1.0,2.0,1.0,0.0,0.0,0.0,1.0,2.0
4,True,End of eviction moratorium means millions of A...,@OhComfy I agree. The confluence of events rig...,70.0,166.0,15282.0,2194.0,0.0,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]],0.03,...,7.333333,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df = df.drop(columns=['embeddings'])


In [ ]:
#Ovo je nepotrebno
df = df.drop(columns=['total_count'])
df = df.drop(columns=['unique_count'])

In [118]:
df.columns

Index(['majority_target', 'statement', 'tweet', 'followers_count',
       'friends_count', 'favourites_count', 'statuses_count', 'listed_count',
       'BotScore', 'BotScoreBinary', 'cred', 'normalize_influence', 'mentions',
       'quotes', 'replies', 'retweets', 'favourites', 'hashtags', 'URLs',
       'tweet_new', 'tweet_tokens', 'Word count', 'Max word length',
       'Min word length', 'Average word length', 'Present verbs', 'Past verbs',
       'Adjectives', 'Adverbs', 'Pronouns', 'Conjunctions', 'Determiners',
       'Prepositions', 'Digits'],
      dtype='object')

In [119]:
zero_columns = df.columns[(df == 0.0).all()]
print(zero_columns)


Index([], dtype='object')


In [120]:
df.to_csv("df_new.csv")

# Empath

In [ ]:
lexicon = Empath()


empath_scores= []


for i, tweet in enumerate(df['tweet_new']):
    try:
        empath_result = lexicon.analyze(tweet, normalize=True)
        empath_scores.append({'tweet_new': tweet, 'empath_result': empath_result})
    except Exception as e:
        print(f"An error occurred for tweet {i}: {e}")

df_empath_2 = pd.json_normalize(empath_scores)

In [ ]:
df_empath_2.to_csv("df_empath.csv")

In [ ]:
df_empath_2 = [
    "tweet_new", 
    "empath_result.negative_emotion", 
    "empath_result.positive_emotion",
    "empath_result.joy",
    "empath_result.anger",
    "empath_result.fear",
    "empath_result.sadness",
    "empath_result.nervousness",
    "empath_result.anxiety",
    "empath_result.pain",
    "empath_result.sympathy",
    "empath_result.trust"
]

In [ ]:
merged_df = pd.merge(df, df_empath_2, on='tweet_new', how='inner')
merged_df

,majority_target,statement,BinaryNumTarget,tweet,followers_count,friends_count,favourites_count,statuses_count,listed_count,embeddings,...,empath_result.positive_emotion,empath_result.politics,empath_result.violence,empath_result.health,empath_result.war,empath_result.death,empath_result.suffering,empath_result.law,empath_result.valuable,empath_result.giving
0,True,End of eviction moratorium means millions of A...,1.0,@POTUS Biden Blunders - 6 Month Update\n\nInfl...,4262.0,3619.0,34945.0,16423.0,44.0,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]],...,0.000000,0.0000,0.033333,0.00000,0.00000,0.0,0.00000,0.0,0.0,0.033333
1,True,End of eviction moratorium means millions of A...,1.0,@POTUS Biden Blunders - 6 Month Update\n\nInfl...,4262.0,3619.0,34945.0,16423.0,44.0,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]],...,0.000000,0.0000,0.033333,0.00000,0.00000,0.0,0.00000,0.0,0.0,0.033333
2,True,End of eviction moratorium means millions of A...,1.0,@S0SickRick @Stairmaster_ @6d6f636869 Not as m...,1393.0,1621.0,31436.0,37184.0,64.0,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]],...,0.000000,0.0000,0.000000,0.00000,0.00000,0.0,0.00000,0.0,0.0,0.000000
3,True,End of eviction moratorium means millions of A...,1.0,THE SUPREME COURT is siding with super rich pr...,9.0,84.0,219.0,1184.0,0.0,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]],...,0.000000,0.0000,0.000000,0.00000,0.00000,0.0,0.00000,0.0,0.0,0.000000
4,True,End of eviction moratorium means millions of A...,1.0,@POTUS Biden Blunders\n\nBroken campaign promi...,4262.0,3619.0,34945.0,16423.0,44.0,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]],...,0.000000,0.0625,0.031250,0.03125,0.03125,0.0,0.03125,0.0,0.0,0.031250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274070,False,Joe Bidens great-grandfather Joseph J. Biden w...,0.0,"@ChevyChaseToGo ""Joe Bidens great-grandfather ...",286.0,2501.0,28377.0,7617.0,1.0,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]],...,0.000000,0.0000,0.000000,0.00000,0.00000,0.0,0.00000,0.0,0.0,0.000000
274071,False,Joe Bidens great-grandfather Joseph J. Biden w...,0.0,"@ChevyChaseToGo ""Joe Bidens great-grandfather ...",286.0,2501.0,28377.0,7617.0,1.0,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]],...,0.000000,0.0000,0.000000,0.00000,0.00000,0.0,0.00000,0.0,0.0,0.000000
274072,False,Joe Bidens great-grandfather Joseph J. Biden w...,0.0,"@ChevyChaseToGo ""Joe Bidens great-grandfather ...",286.0,2501.0,28377.0,7617.0,1.0,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]],...,0.000000,0.0000,0.000000,0.00000,0.00000,0.0,0.00000,0.0,0.0,0.000000
274073,False,Joe Bidens great-grandfather Joseph J. Biden w...,0.0,@JoeBiden Facts are Bidens VP Kamala Harris Gr...,2.0,45.0,16.0,509.0,0.0,[[0 1 0 0 0 0 1 0 0 0 0 0 1 1 0 0 1]],...,0.133333,0.0000,0.000000,0.00000,0.00000,0.0,0.00000,0.0,0.0,0.000000
